# __Activity 4__
## Rachel Nana, December 2018 - January 2019

__Importing Packages__

In [1]:
import pandas as pd
import sklearn
import numpy as np

__Importing Telco Data__

In [2]:
telco = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv');
telco.drop('customerID', axis=1, inplace=True)

__Data Prep__

Variables

In [3]:
X = telco.loc[:, telco.columns != 'Churn'].copy()
y = telco.loc[:, 'Churn'].copy()

Split The Data

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [6]:
X_train[:5]

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
2142,Female,0,No,Yes,21,Yes,No,DSL,Yes,No,Yes,No,No,Yes,One year,No,Mailed check,64.85,1336.8
1623,Female,0,No,No,54,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),97.20,5129.45
6074,Male,0,Yes,No,1,No,No phone service,DSL,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,23.45,23.45
1362,Male,0,No,No,4,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.20,237.95
6754,Male,0,No,Yes,0,Yes,Yes,DSL,Yes,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,


Replace Blanks

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin

In [8]:
class ReplaceBlanks(TransformerMixin):    
    def transform(self, X, y=None, **kwargs):
        try:
            new = X.replace(" ", np.nan, inplace=True)
            return new
        except Exception as err:
            print('ReplaceBlanks.transform(): {}'.format(err))
        return X

    def fit(self, X, y=None, **kwargs):
        return self

In [9]:
RB = ReplaceBlanks()
RB.fit_transform(X_train)

C:\Users\rfowl\Anaconda3\lib\site-packages\pandas\core\frame.py:3798: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


TotalCharges Data Type

In [10]:
X_train['TotalCharges'] = X_train['TotalCharges'].astype('float')

C:\Users\rfowl\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Value Scaler

In [11]:
from sklearn_pandas import DataFrameMapper

In [12]:
from sklearn import impute

In [13]:
value_scaler = DataFrameMapper([
    (['TotalCharges'], [sklearn.impute.SimpleImputer(missing_values=np.nan, strategy='median'),
                        sklearn.preprocessing.StandardScaler()]),
    (['tenure'], sklearn.preprocessing.StandardScaler()),
    (['MonthlyCharges'], sklearn.preprocessing.StandardScaler())
], df_out=True)

In [14]:
scaled_values = value_scaler.fit_transform(X_train)

C:\Users\rfowl\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\rfowl\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [15]:
scaled_values[:5]

,TotalCharges,tenure,MonthlyCharges
2142,-0.421357,-0.465683,-0.000474
1623,1.255883,0.885537,1.074754
6074,-1.002166,-1.284605,-1.376499
1362,-0.907306,-1.161766,0.177346
6754,-0.388632,-1.325551,-0.098524


In [16]:
np.any(np.isnan(X_train['TotalCharges']))

True

In [17]:
np.any(np.isnan(scaled_values))

False

In [18]:
scaled_values.dtypes

TotalCharges      float64
tenure            float64
MonthlyCharges    float64
dtype: object

Indicators

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin

In [20]:
class CreateInds(BaseEstimator, TransformerMixin):    
    def transform(self, X, y=None, **kwargs):
        try:
            new_X = pd.get_dummies(X, drop_first=True)
            
            return new_X
        except Exception as err:
            print('CreateInds.transform(): {}'.format(err))
        return (X, y)

    def fit(self, X, y=None, **kwargs):
        return self

In [21]:
from sklearn_pandas import gen_features

In [22]:
feature_def = gen_features(
    columns=['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod'],
    classes=[CreateInds]
)

In [23]:
feature_def.append(('SeniorCitizen', None))

In [24]:
telco_inds = DataFrameMapper(feature_def, df_out=True)

In [25]:
indicators = telco_inds.fit_transform(X_train)

In [26]:
indicators[:5]

,gender,Partner,Dependents,PhoneService,MultipleLines_0,MultipleLines_1,InternetService_0,InternetService_1,OnlineSecurity_0,OnlineSecurity_1,...,StreamingTV_1,StreamingMovies_0,StreamingMovies_1,Contract_0,Contract_1,PaperlessBilling,PaymentMethod_0,PaymentMethod_1,PaymentMethod_2,SeniorCitizen
2142,0,0,1,1,0,0,0,0,0,1,...,0,0,1,1,0,0,0,0,1,0
1623,0,0,0,1,0,1,1,0,0,0,...,1,0,1,0,1,1,0,0,0,0
6074,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1362,1,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
6754,1,0,1,1,0,1,0,0,0,1,...,0,0,0,0,1,1,0,0,0,0


In [27]:
X_train[:5]

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
2142,Female,0,No,Yes,21,Yes,No,DSL,Yes,No,Yes,No,No,Yes,One year,No,Mailed check,64.85,1336.80
1623,Female,0,No,No,54,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),97.20,5129.45
6074,Male,0,Yes,No,1,No,No phone service,DSL,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,23.45,23.45
1362,Male,0,No,No,4,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.20,237.95
6754,Male,0,No,Yes,0,Yes,Yes,DSL,Yes,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,NaN


In [28]:
from sklearn.pipeline import FeatureUnion

In [29]:
mapper_union = FeatureUnion([
  ("scaled_values", scaled_values),
  ("telco_inds", telco_inds)
])

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().